## Limpeza, padronização e transformação dos CSV "Mortalidade_Geral"

In [ ]:
import pandas as pd
import numpy as np
import duckdb
from pathlib import Path
import warnings

# Suprimir warnings
warnings.filterwarnings('ignore')

# Configurar caminhos
RAW_PATH = Path('../data/raw')
PROCESSED_PATH = Path('../data/processed')
PROCESSED_PATH.mkdir(exist_ok=True, parents=True)


COLUNAS_INTERESSE = ['CODMUNRES', 'CAUSABAS', 'DTOBITO']

print("Ambiente configurado.")

Como o arquivo é muito grande, essa função lê o arquivo de mortalidade em pedaços de 100.000 linhas, filtra o que é cardio e dps agrega, para não correr o risco de maquina fracas como a minha não consiga terminar o processo. 

In [ ]:
def processar_mortalidade(ano):
    print(f'\n=== Processando Mortalidade {ano} ===')
    
    # Tenta padroes de nome comuns do Datasus
    file_path = RAW_PATH / f'mortalidade_geral{ano}.csv'
    
    if not file_path.exists():
        print(f'[ERRO] Arquivo nao encontrado: {file_path}')
        return pd.DataFrame()

    print(f'Lendo arquivo: {file_path}')

    chunks = []
    
    # Ler o arquivo em pedacos (chunks) de 100 mil linhas
    try:
        iterator = pd.read_csv(
            file_path, 
            sep=';', 
            usecols=COLUNAS_INTERESSE,
            encoding='latin1',
            dtype={'CODMUNRES': str, 'CAUSABAS': str},
            chunksize=100000,
            low_memory=False
        )
        
        for i, chunk in enumerate(iterator):
            # 1. Filtrar apenas Doencas Cardiovasculares
            # Capitulo IX do CID-10 vai de I00 a I99. 
            # Entao pegamos tudo que comeca com 'I'
            chunk = chunk.dropna(subset=['CAUSABAS'])
            chunk_cardio = chunk[chunk['CAUSABAS'].str.startswith('I', na=False)].copy()
            
            if not chunk_cardio.empty:
                # 2. Padronizar Municipio (6 digitos)
                chunk_cardio['codmun'] = chunk_cardio['CODMUNRES'].str.slice(0, 6)
                
                # 3. Contar obitos por municipio neste pedaco
                # Agrupamos agora para reduzir o tamanho dos dados drasticamente
                resumo_chunk = chunk_cardio.groupby('codmun').size().reset_index(name='obitos')
                chunks.append(resumo_chunk)
                
    except Exception as e:
        print(f'[ERRO] Falha ao ler arquivo: {e}')
        return pd.DataFrame()

    # Consolidar todos os pedacos do ano
    if chunks:
        df_ano = pd.concat(chunks, ignore_index=True)
        
        # Re-agrupar para somar os totais dos chunks
        df_ano = df_ano.groupby('codmun')['obitos'].sum().reset_index()
        df_ano.rename(columns={'obitos': 'mortes_cardio'}, inplace=True)
        
        df_ano['ano'] = int(ano)
        
        print(f'Total de obitos cardio processados em {ano}: {df_ano["mortes_cardio"].sum()}')
        return df_ano
    else:
        return pd.DataFrame()

Consolidação 

In [ ]:
dfs_mortalidade = []

# Processar os anos definidos
for ano in [2007, 2010, 2015]:
    df_temp = processar_mortalidade(ano)
    if not df_temp.empty:
        dfs_mortalidade.append(df_temp)

# Juntar tudo
if dfs_mortalidade:
    df_final_mort = pd.concat(dfs_mortalidade, ignore_index=True)
    
    # Filtrar codigos de municipio invalidos (tamanho diferente de 6)
    df_final_mort = df_final_mort[df_final_mort['codmun'].str.len() == 6]
    
    print('\n=== Dataset Mortalidade Consolidado ===')
    print(f'Shape: {df_final_mort.shape}')
    print(df_final_mort.head())
    
    # Salvar Parquet
    output_file = PROCESSED_PATH / 'mortalidade_cardio_clean.parquet'
    df_final_mort.to_parquet(output_file, index=False)
    print(f'[SUCESSO] Salvo em: {output_file}')
else:
    print('[ERRO] Nenhum dado processado.')

SQL só para visualizar se está tudo nos conformes kkkk 

In [ ]:
print('\n=== Validacao com DuckDB ===')

con = duckdb.connect()
con.register('tb_mortalidade', df_final_mort)

# 1. Resumo por Ano (Verificar se tem dados para todos os anos)
query_resumo = """
SELECT 
    ano, 
    COUNT(*) as qtd_municipios_com_obito,
    SUM(mortes_cardio) as total_obitos_brasil,
    AVG(mortes_cardio)::INT as media_obitos_cidade,
    MAX(mortes_cardio) as max_obitos_cidade
FROM tb_mortalidade
GROUP BY ano
ORDER BY ano
"""
print('\n[SQL] Resumo de Mortalidade Cardiovascular:')
print(con.execute(query_resumo).df())

# 2. Checagem Top 5 Cidades (Validacao de consistencia - SP e Rio devem liderar)
query_top = """
SELECT codmun, mortes_cardio
FROM tb_mortalidade
WHERE ano = 2015
ORDER BY mortes_cardio DESC
LIMIT 5
"""
print('\n[SQL] Top 5 Cidades com mais obitos cardio (2015):')
print(con.execute(query_top).df())